In [58]:
import py_entitymatching as em
import pandas as pd
import os,sys
import math

In [59]:
## Reading csv tables into pandas dataframe

A=em.read_csv_metadata('Data/csv/yelp_list.csv')
A['ID'] = range(0,len(A))
em.set_key(A,'ID')


B=em.read_csv_metadata('Data/csv/zomato_list.csv')
B['ID'] = range(0,len(B))
em.set_key(B,'ID')

True

In [60]:
##Data Cleaning1: Converting  price range to an absolute number. Will be applied for yelp list 


def clean_price_range( price_str ):

    avg_price = ''
    
    if not isinstance(price_str,basestring)  and  math.isnan(price_str):
        return price_str

    str2 = price_str.replace('$','')
        
    if '-' in str2:
        num_l = int(str2.split('-')[0])
        num_r = int(str2.split('-')[1])
        num_avg = (num_l+num_r)/2
        avg_price = '$' + str(num_avg)

    else:
        str_word = str2.split()[0].lower()
        num_price  = int(str2.split()[1])
        
        if str_word == 'above':
            num_avg = int(num_price + num_price/2)
            avg_price = '$' + str(num_avg)

        else:
            num_avg = num_price/2
            avg_price = '$' + str(num_avg)
            
    return avg_price

In [61]:
##Data Cleaning2: Removing "reviews" string from num_reviews column. Will be applied for zomato list


def clean_num_reviews(review_string) :
      return int(review_string.split()[0])

In [62]:
##Data Cleaning3: Removing "restaurant" if it is the last word in restaurant name. to prevent inconsistentcy.Will be applied
## for both lists


def clean_name( name_str):

    name_lower = name_str.lower()

    name_2 = name_lower.split()[-1]
    
    if name_2 == 'restaurant':
        name_lower = name_lower.replace('restaurant','')
        
    return name_lower

In [63]:
##Data Cleaning4: Removing quotes and shortening some words in address.Will be applied for both zomato list
## only zomato 

def clean_address( address ):
    addr_2 = address.replace('\"','')
    addr_3 = addr_2.replace(',','')
    if "Street" in addr_3:
        addr_3 = addr_3.replace('Street','St')
    if "Boulevard" in addr_3:
        addr_3 = addr_3.replace('Boulevard','Blvd')
        
    return addr_3

In [64]:
def clean_rating_value(ratingValue):
    return str(ratingValue)

In [65]:
########## DATA CLEANING ################

A['price_range'] = A['price_range'].apply( clean_price_range )
A['name'] = A['name'].apply( clean_name )
A['ratingValue'] = A['ratingValue'].apply(clean_rating_value)



B['name'] = B['name'].apply( clean_name )
B['address'] = B['address'].apply( clean_address )
B['number_of_reviews'] = B['number_of_reviews'].apply( clean_num_reviews )
B['ratingValue'] = B['ratingValue'].apply(clean_rating_value)

In [66]:
#To find missing values 
len(A.ID) - A.count()

number_of_reviews     0
price_range          44
ratingValue           0
name                  0
address               0
ID                    0
dtype: int64

In [67]:
len(B.ID) - B.count()

number_of_reviews    0
price_range          0
ratingValue          0
name                 0
address              0
ID                   0
dtype: int64

In [68]:
block_f = em.get_features_for_blocking(A,B)

In [69]:
block_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,number_of_reviews_number_of_reviews_exm,number_of_reviews,number_of_reviews,None,None,exact_match,<function number_of_reviews_number_of_reviews_exm at 0x7f729912e0c8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,number_of_reviews_number_of_reviews_anm,number_of_reviews,number_of_reviews,None,None,abs_norm,<function number_of_reviews_number_of_reviews_anm at 0x7f729912e140>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,number_of_reviews_number_of_reviews_lev_dist,number_of_reviews,number_of_reviews,None,None,lev_dist,<function number_of_reviews_number_of_reviews_lev_dist at 0x7f729912e2a8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,number_of_reviews_number_of_reviews_lev_sim,number_of_reviews,number_of_reviews,None,None,lev_sim,<function number_of_reviews_number_of_reviews_lev_sim at 0x7f729912e758>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,price_range_price_range_lev_dist,price_range,price_range,None,None,lev_dist,<function price_range_price_range_lev_dist at 0x7f729912e320>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,price_range_price_range_lev_sim,price_range,price_range,None,None,lev_sim,<function price_range_price_range_lev_sim at 0x7f729912eb18>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,price_range_price_range_jar,price_range,price_range,None,None,jaro,<function price_range_price_range_jar at 0x7f729912e7d0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,price_range_price_range_jwn,price_range,price_range,None,None,jaro_winkler,<function price_range_price_range_jwn at 0x7f729912e938>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,price_range_price_range_exm,price_range,price_range,None,None,exact_match,<function price_range_price_range_exm at 0x7f729912e848>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,price_range_price_range_jac_qgm_3_qgm_3,price_range,price_range,qgm_3,qgm_3,jaccard,<function price_range_price_range_jac_qgm_3_qgm_3 at 0x7f729912e8c0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [72]:
##Rule based on restaurant name similarity, 0.80 score is derived from debugging blocking output
rb1 = em.RuleBasedBlocker()
rule1 = 'name_name_lev_sim(ltuple,rtuple) < 0.80'
rb1.add_rule(rule1, block_f )

##Rule based on address similarity
rb2 = em.RuleBasedBlocker()
rule2 = 'address_address_jac_qgm_3_qgm_3(ltuple,rtuple) < 0.7'
rb2.add_rule(rule2,block_f)

'_rule_0'

In [73]:
## To block based on zipcode equivalence

def zipcode_match(x, y):
    # x, y will be of type pandas series
    
    # get address attribute
    x_address = x['address']
    y_address = y['address']
    
    # get the zipcode
    x_split, y_split = x_address.split(), y_address.split()
    x_zipcode = x_split[len(x_split) - 1]
    y_zipcode = y_split[len(y_split) - 1]
    
    # check if the zipcode match
    if x_zipcode != y_zipcode:
        return True
    else:
        return False

In [74]:
# Instantiate blackbox blocker
bb = em.BlackBoxBlocker()

# Set the black box function
bb.set_black_box_function(zipcode_match)

In [ ]:
## Blocking Pipeline- First block based on zip code then block based on address similarity##

C1 = bb.block_tables(A,B,l_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],r_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],n_jobs=-1)

C2 = rb2.block_candset(C1,n_jobs=-1)

###Direct name blocker###
C3 = rb1.block_tables(A,B,l_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],r_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],n_jobs=-1)

## Direct Address blocker not enabled##
#C2 = rb2.block_tables(A,B,l_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],r_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'])

C4 = em.combine_blocker_outputs_via_union([C2, C3])

0%                          100%
[############                  ] | ETA: 00:08:59 | ETA: 00:08:35 | ETA: 00:08:13 | ETA: 00:08:23 | ETA: 00:08:11 | ETA: 00:07:53 | ETA: 00:07:59 | ETA: 00:07:30 | ETA: 00:07:02 | ETA: 00:06:50 | ETA: 00:06:42 | ETA: 00:06:33

In [17]:
## Debugging blocker to make sure true positives are not left out in candidate set
dbg = em.debug_blocker(C4,A,B,output_size=400)

In [18]:
dbg

,_id,similarity,ltable_ID,rtable_ID,ltable_number_of_reviews,ltable_price_range,ltable_ratingValue,ltable_name,ltable_address,rtable_number_of_reviews,rtable_price_range,rtable_ratingValue,rtable_name,rtable_address
0,0,0.538462,2365,1834,16,$5,4.0,beto’s mexican,3518 E 7th St Austin TX 78702,19,$40,3.2,angie's mexican,1307 E 7th St Austin TX 78702
1,1,0.538462,2433,563,367,$45,4.0,place pigalle,81 Pike St Seattle WA 98101,43,$110,3.8,maximilien,Pike Place Market 81 Pike St Seattle WA 98101
2,2,0.538462,1391,1606,1396,$5,4.5,torchy’s tacos,1311 S 1st St Austin TX 78704,17,$40,3.7,mellizoz tacos,1503 S 1St St Austin TX 78704
3,3,0.529412,1927,1107,225,$20,3.5,maudie’s milagro,3801 N Capital Of Texas Hwy Austin TX 78746,11,$65,4.0,360 uno,3801 N Capital of Texas Hwy Suite G100 Austin TX 78746
4,4,0.529412,1717,1192,1013,$5,4.0,pho y #1,1660 E Capitol Expy San Jose CA 95121,2,$70,3.8,pho y #1 noodle house,1660 East Capitol Expressway San Jose CA 95121
5,5,0.500000,1652,1483,266,$20,4.0,bocaphê,222 Lafayette St New York NY 10012,17,$70,4.1,cuba,222 Thompson St New York NY 10012
6,6,0.500000,1032,727,423,$20,4.0,55 south,55 S 1st St San Jose CA 95113,4,$70,3.4,nemea,96 South 1st St San Jose CA 95113
7,7,0.500000,2143,159,212,$20,4.0,balzem,202 Mott St New York NY 10012,212,$140,4.5,balthazar,80 Spring St New York NY 10012
8,8,0.500000,1219,1251,26,$20,4.5,treehaus mima,470 W 42nd St New York NY 10036,26,$15,4.2,schmackary's,362 W 45th St New York NY 10036
9,9,0.500000,2997,18,821,$20,3.0,the cheesecake factory,925 Blossom Hill Road San Jose CA 95123,8,$80,4.3,bj's,Westfield Oakridge 925 Blossom Hill Road San Jose CA 95123


In [19]:
##Writing final candidate set to CSV file
em.to_csv_metadata(C4,'./C4_output.csv')

True

In [ ]:
G = em.label_table(S, 'gold_labels')

In [ ]:
G = em.label_table(S, label_col_name='gold_labels')

In [ ]:
train_test = em.split_train_test(G, train_proportion=0.5)

In [ ]:
devel_set = train_test['train']

In [ ]:
eval_set = train_test['test']

In [ ]:
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()

In [ ]:
atypes1 = em.get_attr_types(A) # don't need, if atypes1 exists from blocking step
atypes2 = em.get_attr_types(B) # don't need, if atypes2 exists from blocking step

match_c = em.get_attr_corres(A, B)

In [ ]:
match_f = em.get_features_for_matching(A, B)

In [ ]:
em._match_t

In [ ]:
em._match_s

In [ ]:
em._atypes1

In [37]:
em._atypes2

{'ID': 'numeric',
 '_table':      number_of_reviews price_range ratingValue                         name  \
 0                   24         $20         4.6         la victoria taqueria   
 1                   19         $40         4.6                  the counter   
 2                   44         $20         4.9           falafel's drive-in   
 3                   17         $40         4.4               new krung thai   
 4                   28         $70         4.1               original joe's   
 5                    6         $30         4.2                    adelita's   
 6                   14         $65         4.0      maggiano's little italy   
 7                   26         $40         4.4                blue rock bbq   
 8                   16         $43         4.2              zeni ethiopian    
 9                    9         $35         4.3              sam's bar-b-que   
 10                  14         $65         4.0                  amber india   
 11         

In [38]:
em._match_c

{'corres': [('number_of_reviews', 'number_of_reviews'),
  ('price_range', 'price_range'),
  ('ratingValue', 'ratingValue'),
  ('name', 'name'),
  ('address', 'address'),
  ('ID', 'ID')],
 'ltable':       number_of_reviews price_range  ratingValue  \
 0                   125         $20          5.0   
 1                  1041         $91          4.0   
 2                   146         $20          4.5   
 3                   266         $20          4.5   
 4                   159         $20          4.5   
 5                   287         $20          4.5   
 6                  1210         $20          4.5   
 7                   890         $20          4.0   
 8                   772         $45          4.5   
 9                   576         $20          4.5   
 10                  192         $45          4.0   
 11                  392         $20          4.0   
 12                  165         $20          4.5   
 13                  103         $20          5.0   
 14     

In [ ]:
H = em.extract_feature_vecs(G, match_f, attrs_before=['title'], attrs_after=['gold_labels'])

In [ ]:
H = em.extract_feature_vecs(G, features=match_f, attrs_before=['title'], attrs_after=['gold_labels'])

In [ ]:
H.fillna(value=0, inplace=True)

In [ ]:
dt = em.DTMatcher(max_depth=5)

In [ ]:
dt.fit(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='gold_labels')

In [ ]:
dt.predict(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='predicted_labels', append=True, inplace=True)

In [ ]:
train_test = em.split_train_test(devel, 0.5)

In [ ]:
train, test = train_test['train'], train_test['test']

In [ ]:
em.vis_debug_dt(dt, train, test, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='gold_labels')